In [25]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.fft as fft
import pandas as pd
import datetime as dt

In [38]:
x = pd.read_csv('Train.csv')
# print(x)
time = x['Datetime']
first = dt.datetime.strptime(time[0],'%d-%m-%Y %H:%M')
last = dt.datetime.strptime(time[len(time)-1],'%d-%m-%Y %H:%M')
time_interval =  (last - first).days
# print(time_interval)
samples =  x['Count']
N = len(samples)
frecv = N/time_interval
print(frecv)

24.031537450722734
